In [1]:
import numpy as np
import math
import xarray as xr
import dask
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import netCDF4
import pandas as pd
from sklearn.cluster import KMeans
from scipy.ndimage.filters import uniform_filter1d

In [2]:
path = '/DFS-L/DATA/pritchard/gmooers/Workflow/MAPS/SPCAM/100_Days/New_SPCAM5/archive/TimestepOutput_Neuralnet_SPCAM_216/atm/hist/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2009-01-1[56789]*'
real_ds = xr.open_mfdataset(path)
Q_CRM = real_ds.CRM_QV_LON_0e_to_360e_LAT_20s_to_20n
W_CRM = real_ds.CRM_W_LON_0e_to_360e_LAT_20s_to_20n
T_CRM = real_ds.CRM_T_LON_0e_to_360e_LAT_20s_to_20n
Precip_CRM = real_ds.CRM_PREC_LON_0e_to_360e_LAT_20s_to_20n

Q_CRM = xr.DataArray.squeeze(Q_CRM).values
T_CRM = xr.DataArray.squeeze(T_CRM).values
W_CRM = xr.DataArray.squeeze(W_CRM)
Scalar_W_CRM = xr.DataArray.squeeze(W_CRM[:,18,:,:,:]).values
Precip_CRM = xr.DataArray.squeeze(Precip_CRM).values

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/export/home/gmooers/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-71e53d16075c>", line 8, in <module>
    Q_CRM = xr.DataArray.squeeze(Q_CRM).values
  File "/export/home/gmooers/miniconda3/lib/python3.7/site-packages/xarray/core/dataarray.py", line 410, in values
    return self.variable.values
  File "/export/home/gmooers/miniconda3/lib/python3.7/site-packages/xarray/core/variable.py", line 392, in values
    return _as_array_or_item(self._data)
  File "/export/home/gmooers/miniconda3/lib/python3.7/site-packages/xarray/core/variable.py", line 213, in _as_array_or_item
    data = np.asarray(data)
  File "/export/home/gmooers/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py", line 85, in asarray
    return array(a, dtype, copy=False, order=order)
  File "/export/home/gmooers/miniconda3/lib/python3.7/sit

KeyboardInterrupt: 

In [22]:
others = netCDF4.Dataset("/fast/gmooers/Raw_Data/extras/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2009-01-01-72000.nc")
plev = np.array(others.variables['lev'])
ps = np.array(others.variables['PS'])
g = 9.81 #m/s^2
hyai = np.array(others.variables['hyai'])
hybi = np.array(others.variables['hybi'])
#print(hyai.shape)
#print(hyai)
cp = 1004.0
L = 2256000.0
PS = 1e5
P0 = 1e5
P = P0*hyai+PS*hybi # Total pressure [Pa]
dp = P[1:]-P[:-1] # Differential pressure [Pa]
Scalar_Q_CRM = np.nansum(Q_CRM*dp[None, :, None, None, None]*L/g, axis=1)
Scalar_T_CRM = np.nansum(T_CRM*dp[None, :, None, None, None]*cp/g, axis=1)

In [23]:
Q_Reshape = np.zeros(shape=(len(Scalar_Q_CRM)*len(Scalar_Q_CRM[0][0])*len(Scalar_Q_CRM[0][0][0]),128))
Q_Reshape[:,:] = np.nan
T_Reshape = np.zeros(shape=(len(Scalar_T_CRM)*len(Scalar_T_CRM[0][0])*len(Scalar_T_CRM[0][0][0]),128))
T_Reshape[:,:] = np.nan
W_Reshape = np.zeros(shape=(len(Scalar_W_CRM)*len(Scalar_W_CRM[0][0])*len(Scalar_W_CRM[0][0][0]),128))
W_Reshape[:,:] = np.nan
Precip_Reshape = np.zeros(shape=(len(Precip_CRM)*len(Precip_CRM[0][0])*len(Precip_CRM[0][0][0]),128))
Precip_Reshape[:,:] = np.nan

In [24]:
count = 0
for i in range(len(Scalar_Q_CRM)):
    for j in range(len(Scalar_Q_CRM[0][0])):
        for k in range(len(Scalar_Q_CRM[0][0][0])):
            Q_Reshape[count,:] = Scalar_Q_CRM[i,:,j,k]
            T_Reshape[count,:] = Scalar_T_CRM[i,:,j,k]
            W_Reshape[count,:] = Scalar_W_CRM[i,:,j,k]
            Precip_Reshape[count,:] = Precip_CRM[i,:,j,k]
            count = count+1

In [25]:
q_train = Q_Reshape[:int(4*len(Q_Reshape)/5),:]
q_test = Q_Reshape[int(4*len(Q_Reshape)/5):,:]

t_train = T_Reshape[:int(4*len(T_Reshape)/5),:]
t_test = T_Reshape[int(4*len(T_Reshape)/5):,:]

w_train = W_Reshape[:int(4*len(W_Reshape)/5),:]
w_test = W_Reshape[int(4*len(W_Reshape)/5):,:]

precip_train = Precip_Reshape[:int(4*len(Precip_Reshape)/5),:]
precip_test = Precip_Reshape[int(4*len(Precip_Reshape)/5):,:]

In [26]:
q_Shuffled = shuffle(q_train)
t_Shuffled = shuffle(t_train)
w_Shuffled = shuffle(w_train)
precip_Shuffled = shuffle(precip_train)

In [27]:
Q_rescaled_train = np.interp(q_Shuffled, (q_Shuffled.min(), q_Shuffled.max()), (0, +1))
Q_rescaled_test = np.interp(q_test, (q_Shuffled.min(), q_Shuffled.max()), (0, +1))

T_rescaled_train = np.interp(t_Shuffled, (t_Shuffled.min(), t_Shuffled.max()), (0, +1))
T_rescaled_test = np.interp(t_test, (t_Shuffled.min(), t_Shuffled.max()), (0, +1))

W_rescaled_train = np.interp(w_Shuffled, (w_Shuffled.min(), w_Shuffled.max()), (0, +1))
W_rescaled_test = np.interp(w_test, (w_Shuffled.min(), w_Shuffled.max()), (0, +1))

Precip_rescaled_train = np.interp(precip_Shuffled, (precip_Shuffled.min(), precip_Shuffled.max()), (0, +1))
Precip_rescaled_test = np.interp(precip_test, (precip_Shuffled.min(), precip_Shuffled.max()), (0, +1))

In [28]:
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_Q_Training.npy', Q_rescaled_train)
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_Q_Test.npy', Q_rescaled_test)
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_Q_Max_Scalar.npy', q_train.max())
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_Q_Min_Scalar.npy', q_train.min())

np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_T_Training.npy', T_rescaled_train)
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_T_Test.npy', T_rescaled_test)
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_T_Max_Scalar.npy', t_train.max())
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_T_Min_Scalar.npy', t_train.min())

np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_W_Training.npy', W_rescaled_train)
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_W_Test.npy', W_rescaled_test)
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_W_Max_Scalar.npy', w_train.max())
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_W_Min_Scalar.npy', w_train.min())

np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_Precip_Training.npy', Precip_rescaled_train)
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_Precip_Test.npy', Precip_rescaled_test)
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_Precip_Max_Scalar.npy', precip_train.max())
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_Precip_Min_Scalar.npy', precip_train.min())